# NLP

## Explore Tiny Shakespear dataset

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("Length of the dataset in characters: ", len(text))

Length of the dataset in characters:  1115393


In [91]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"Number of unique characters: {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Number of unique characters: 65


## Tokenizer

In [10]:
stoi = { ch:i for i,ch in enumerate(chars) } # string to integer
itos = { i:ch for i,ch in enumerate(chars) } # integer to string
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [14]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## Dataset Split

In [17]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [25]:
block_size = 8 # or context length
train_data[:block_size +1 ]
# It is nine characters because then the model can predict the next character 8 times

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [26]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When the input is {context} the target: {target}")

When the input is tensor([18]) the target: 47
When the input is tensor([18, 47]) the target: 56
When the input is tensor([18, 47, 56]) the target: 57
When the input is tensor([18, 47, 56, 57]) the target: 58
When the input is tensor([18, 47, 56, 57, 58]) the target: 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [27]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----
when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 43, 54, 1] the target: 47
when input is [49, 43, 

In [56]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # because no targets -> Loss is None -> logits still have to be permuted -> becomes (B, C)
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1) # Es gibt ja jetzt für jeden Token eine softmax wahrscheinlichkeit
            # torch.multinomial nimmt jetzt zufällig einen dieser Token with respect to their wahrscheinlichkeit
            # es ist also am wahrscheinlichsten, dass der Token mit der höchsten probability genommen wird, aber es können auch
            # unwahrscheinlichere gewählt werden, was die Ausgabe natürlicher macht
            # zum beispiel: "Die Katze liegt auf dem " -> Sofa, Bett, Boden -> Greedy würde immer das selbe wählen
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
        

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
#print(logits.shape) # get the predictions for every 
#print(loss)

In [57]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long),max_new_tokens = 100)[0].tolist())) # 0 is the new line character


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [59]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [85]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4394798278808594


In [90]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long),max_new_tokens = 300)[0].tolist()))


I me:


my id, s
S:

Mayo, m.
BUTHERO:
See cect borenghis.
Mory o hisip woow walim,
CULBUERDe ce tow hefrwary theds
thto y, w t an and:
Fr ik's,
FF ldot gofrtarrurndse mbashech.
Aprixeeave, ok.
S al tenowory m:
Fithen t cot sut d thaiday, uisitry whesougus sllle igu h w vevoshet olnje ndous; medewil
